# DP

In [193]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
from numba import jit, float64
from numba.experimental import jitclass
import quantecon as qe
from quantecon.distributions import BetaBinomial


def get_q_w(n, a, b, w_min, w_max):
    q = BetaBinomial(n, a, b).pdf()
    w = np.linspace(w_min, w_max, n+1)
    return q, w 


n, a, b, w_min, w_max = 50, 200, 100, 10, 60                    
q_default, w_default = get_q_w(n, a, b, w_min, w_max) 


mccall_data = [
    ('c', float64),      # unemployment compensation
    ('beta', float64),      # discount factor
    ('w', float64[:]),   # array of wage values, w[i] = wage at state i
    ('q', float64[:])    # array of probabilities
]


@jitclass(mccall_data)
class McCallModel:

    def __init__(self, c=25, beta=0.99, w=w_default, q=q_default):

        self.c, self.beta = c, beta
        self.w, self.q = w_default, q_default

    def state_action_values(self, i, v):
        """
        The values of state-action pairs.
        """
        # Simplify names
        c, beta, w, q = self.c, self.beta, self.w, self.q
        # Evaluate value for each state-action pair
        # Consider action = accept or reject the current offer
        accept = w[i] / (1 - beta)
        reject = c + beta * np.sum(v * q)

        return np.array([accept, reject])
    
@jit(nopython=True)
def compute_reservation_wage(mcm,
                             max_iter=500,
                             tol=1e-6):

    # Simplify names
    c, beta, w, q = mcm.c, mcm.beta, mcm.w, mcm.q

    # == First compute the value function == #

    n = len(w)
    v = w / (1 - beta)          # initial guess
    v_next = np.empty_like(v)
    i = 0
    error = tol + 1
    while i < max_iter and error > tol:

        for i in range(n):
            v_next[i] = np.max(mcm.state_action_values(i, v))

        error = np.max(np.abs(v_next - v))
        i += 1

        v[:] = v_next  # copy contents into v

    # == Now compute the reservation wage == #

    return (1 - beta) * (c + beta * np.sum(v * q))


mcm = McCallModel()
reservation_wage = compute_reservation_wage(mcm)

# RL

In [187]:
import gym
import numpy as np

In [190]:
class McCallSearch(gym.Env):
    
    # n: num of states
    # a, b: params of the betabinomial distribution
    # w_min, w_max: wage lower and upper bound
    # c: unemployment compensation
    # beta: discount factor
    def __init__(self, n=50, a=200, b=100, w_min=10, w_max=60, c=25, beta=0.99):
        from quantecon.distributions import BetaBinomial
        self.n = n
        self.q = BetaBinomial(n, a, b).pdf()
        self.w = np.linspace(w_min, w_max, n+1)
        self.c = c
        self.beta = beta
        # the last state (n+1) is the terminial state
        self.observation_space = gym.spaces.Box(low=0, high=n+1, shape=(1,), dtype=np.float32)
        self.action_space = gym.spaces.Discrete(2)
        
    def sample_space(self):
        return np.random.choice(self.n+1, p=self.q)
    
    def reset(self):
        self.state = np.array([self.sample_space()]).astype(np.float32)
        return self.state
    
    def step(self, action):
        done = False
        wage = self.w[int(self.state[0])] if self.state[0] < self.n+1 else 0
        # 0 denotes accepting the offer, 1 denotes rejecting
        if action == 0:
            done = True
            reward = wage / (1 - self.beta)
            self.state = np.array([self.n+1]).astype(np.float32)
        elif action == 1:
            reward = self.c
            self.reset()
        else:
            raise ValueError("Received invalid action={} which is not part of the action space".format(action))
        return self.state, reward, done, {}

## Set Parameters

In [224]:
n, a, b, w_min, w_max = 10, 200, 100, 10, 60
c, beta = 25, 0.60
q, w = get_q_w(n, a, b, w_min, w_max)

mcm = McCallModel(c=c, beta=beta, q=q, w=w)
reservation_wage = compute_reservation_wage(mcm)

env = McCallSearch(n=n, a=a, b=b, w_min=w_min, w_max=w_max, c=c, beta=beta)

from stable_baselines3.common.env_checker import check_env
check_env(env)

# train

In [225]:
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.env_util import make_vec_env

# Instantiate the env
env = McCallSearch(n=n, a=a, b=b, w_min=w_min, w_max=w_max, c=c, beta=beta)
# wrap it
wrapped_env = make_vec_env(lambda: env, n_envs=1)

In [ ]:
# Train the agent
model = DQN('MlpPolicy', wrapped_env, verbose=0, gamma=env.beta).learn(200000)

# compare performance

In [ ]:
test_env = McCallSearch(n=n, a=a, b=b, w_min=w_min, w_max=w_max, c=c, beta=beta)
wages = test_env.w
for i in range(test_env.n + 1):
    action, _ = model.predict(np.array([i]), deterministic=True)
    true_action = 0 if wages[i] > reservation_wage else 1
    print(f'state: {i}, action: {action}, true action: {true_action}')